# Text Rank와 Keybert 비교
- 참고 https://lovit.github.io/nlp/2019/04/30/textrank/

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import itertools
import re
import kss
from textrank import KeysentenceSummarizer
from textrank import KeywordSummarizer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import itertools


from konlpy.tag import Okt


# 한국어를 포함하고 있는 다국어 SBERT load
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

okt = Okt()

In [123]:
# TextRank
def Okt_tokenizer(sent):
    Oktonken = Okt()
    words = Oktonken.phrases(sent)
    return words
    
def getsummarize(txt):
    sents = kss.split_sentences(txt)
    summarizer = KeysentenceSummarizer(
        tokenize = Okt_tokenizer,
        min_sim = 0.5,
        verbose = True
        )
    keysents = summarizer.summarize(sents, topk=5)
    keysents.sort(key = lambda x : x[0])
    return list(itertools.chain(*keysents))[2::3]

def getkeyword_one(txt):
    sents = kss.split_sentences(txt)
    summarizer = KeywordSummarizer(tokenize=Okt_tokenizer, min_count=2, min_cooccurrence=1)
    keywords = summarizer.summarize(sents, topk=20)
    return list(itertools.chain(*keywords))[0::2]

def getkeyword_sentence(txt_list,):
    #sents = kss.split_sentences(txt)
    summarizer = KeywordSummarizer(tokenize=Okt_tokenizer, min_count=2, min_cooccurrence=1)
    keywords = summarizer.summarize(txt_list, topk=20)
    return list(itertools.chain(*keywords))[0::2]

In [2]:
# keybert 함수
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

def keyword_bert(content):
    try:
        result=''
        tokenized_doc = okt.pos(content)
        global stop_word
        #print(tokenized_doc)
        tokenized = ' '.join([w for w,t in tokenized_doc if  t not in ['Verb'] and w not in stop_word])


        count = CountVectorizer().fit([tokenized]) 
        candidates = count.get_feature_names()

        doc_embedding = model.encode([content])
        candidate_embeddings = model.encode(candidates)
        result = ','.join(mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.8))
    
    except Exception as e:
        print(e)
        print(content)
    return result

In [28]:
df = pd.read_parquet('after_preprocessing.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361706 entries, 0 to 361705
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   source    361706 non-null  object
 1   content   361706 non-null  object
 2   postdate  361706 non-null  object
 3   platform  361706 non-null  object
dtypes: object(4)
memory usage: 11.0+ MB


In [87]:
# 블로그 글 하나 기준 Text Rank
txt = df['content'][0]
print(getkeyword_one(txt,20))

['감귤', '체험', '휴애리', '제주', '축제', '하나', '동백', '사진', '포토', '입장', '여기', '조금', '시간', '지금', '자연생활공원', '모습', '한번', '흑돼지', '카페', '부터']


In [27]:
stop_word = ['있는','있고','있으니','입니다.','이렇게','있을','제주특별자치도']
print(keyword_bert(txt))

자연생활공원,비싸네,곤충,노란,발레,휴애리,우산,먹이,어린이,추워서,매화축제,지도,전통,날개,의자,흑돼지,튤립,나무,안녕하세요,빨간


In [36]:
print(keyword_bert(df['content'][0]))

네이버,화룡,스포츠,후회,해수욕장,수칙,소문,우비,원고료,기도,판매,분홍색,적응,유명인,비결,무섭다며,선택,투명카약,삼시세끼,전혀


In [95]:
source=list(total_df['source'].unique())

# Text Rank와 KeyBert비교
- 아르떼뮤지엄 기준 키워드 위에는 KeyBERT 아래에는 TextRank
#### 결과 : TextRank는 KeyBert와 비교했을 때 관광지를 의미하는 키워드가 부족

In [119]:
print(source[0])
for i in range(5):
    print(total_df['keyword'][i])
    print(getkeyword_one(total_df['content'][i],20))
    print()

아르떼뮤지엄
강릉시,즐거움,주차장,평창동계올림픽,자연,중앙시장,경험,청소년,음식,유명해서,디지털,겨울,에너지,동해안,여행,공간,전국,인기,녹색,도시
['강릉', '아르떼뮤지엄', '강원도', '강릉시', '여행', '코스', '대게', '드라이브', '강문해변', '시간', '강원', '주문진읍', '체험', '바다', '주문진', '도깨비', '파도', '설헌', '센터', '도시']

주차장,관람객,함정,아르떼뮤지엄강릉,많은데,미디어,접근성,도시,정원,전시관,타이밍,공간,겨울,가족,외투,서양,소지품,스크린,여행기,풍경
['관람', '사진', '사람', '강릉', '제주', '전시', '명화', '아르떼뮤지엄', '공간', '가든', '작품', '서양', '시간', '때문', '사실', '항상', '가장', '풍경', '다시', '제주도']

입장권,가족여행,바쁘,낭만,여친,정원,인기,블로그,보름달,노란색,체험,금요일,부지런하기도,코끼리,영업,초콜릿,안녕하세요,기념품,제주여행,보라색
['사진', '작품', '제주', '아르떼뮤지엄', '해변', '정말', '헤라', '다른', '아르떼뮤지엄 제주', '아들', '계속', '시간', '다음', '신랑', '입구', '여기', '사파리', '정글', '파도', '파도 정말']

대만족,식당,멸치볶음,공간,제주공항,우정,중독,산책,메뉴판,영업,부모님,은빛,주차장,쌀밥,취향,편했는데요,활용,식사,빨간,실감
['갈치조림', '제주', '아르떼뮤지엄', '맛집', '저희', '근처', '매콤', '식사', '정도', '모두', '식당', '황해', '입안', '영업', '방문', '시간', '가득', '갈치', '가게', '제주 아르떼뮤지엄']

여수꽃돌게장1번가게장맛집이,음식,공룡,박람회,전기차,주차장,아기랑국내여행목포에서,볶음밥,여수오션뷰카페일단,폭풍,인기,월요일,아침,위생장갑,함정,여수게장맛집,열차,동백꽃,자전거,의자
['전라남도', '여수시', '여수', '아주', '의자', '아기', '맛집', '유모차', '

In [124]:
textrank_df= total_df[total_df['source']=='섭지코지'].copy()
textrank_df['text_rank'] = textrank_df['content'].map(getkeyword_one) 

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required by the normalize function.

In [132]:
keyword_df = total_df.groupby('source')['keyword'].sum()
keyword_df

source
9.81 파크       재미있겠더라고요,비행기,파티,고기,여행,디저트,티비,봄날,관광지,커피숍,메뉴,참여,...
가파도           학교,자전거,블로그,비용,영상,수요일,유채꽃,계획,추웠는데,교회,컴백홈,공유,바쁘고...
곶자왈           제주곶자왈도립공원,어려움,공간,걷기,연기,환상의숲,제주도여행,초등생,산방식당,사슴,...
곽지해수욕장        제주시,커피머신,과물,제주흑돼지,두툼,겨울,음식,거리,장식,교육원,목요일,전통,숯불...
관음사           설산,낭만,경험,야생동물,아쉬움,유럽,추위,프로젝트,눈꽃,맛있습니다,어려움,등산로,...
                                    ...                        
협재해수욕장        음식,부드러움,치즈,공유,홍돈,마늘,직행,돼지고기,메뉴판,채소,재료,별미,육질,빈자...
혼인지           평화로운,웨딩사진,전설,제주공항,정원,남녀,산책,위패,체험,공주,컴퓨터,여름,대중교...
화조원           만족감,식당,정원,공간,교육,토끼,위협,구리,제주아이랑체험,펭귄,향토음식,주차장,다...
휴림            극기훈련,서울근교,놀이,깨끗했어요,캠핑장,크리스마스,도서관,계획,관리,판매,오기,공...
휴애리 자연생활공원    꽃다발,메리크리스마스,연인들,축제,비행기,반려견,그리움,공주,아름다웠을,사진찍기,고...
Name: keyword, Length: 100, dtype: object

In [44]:
total_df = pd.read_parquet('append_keyword.parquet') 

In [45]:
source = total_df['source'].unique()

In [130]:
key_dict={}
for s in source:
    key_list = []
    remove_set = {'네이버','제주도','블로그','포스트','제주여행','공간','주차장','여행','서울','메뉴',
                  '제주공항','제주도여행','대한민국','한국','제주','제주시','포스팅','판매','검색','공항',
                 '안녕하세요','경남','부산','강릉','강릉시','월요일','청주','청주시','비행기','마스크','제주국제공항',
                 }
    
    for key in total_df[total_df['source']==s]['keyword']:
        key_list.extend(key.split(','))
    
    
    key_list = [k for k in key_list if k not in remove_set]    
    
    count = Counter(key_list)
    
    key_count = count.most_common(100)
    
    key_dict[s]= [w for w,n in key_count[:100]]

In [101]:
test_df= pd.DataFrame(data=key_dict.items(),columns=['source','keyword'])

In [102]:
def preprocessing(text):
    result = ' '.join(text)
    return result

In [103]:
test_df['keyword']= test_df['keyword'].map(preprocessing)

In [104]:
test_df.to_parquet('top100_keyword.parquet')

In [105]:
test_df

,source,keyword
0,아르떼뮤지엄,미디어아트 미디어 전시관 강릉시 토끼 경험 입장권 거울 음악 박람회 트릭 색칠 영상...
1,산방산,유채꽃 용머리해안 바람 사진 거리 식당 용머리 음식 계획 기도 영업 자연 가격 디저...
2,함덕해수욕장,해수욕장 식당 바람 산책 해변 음식 거리 사진 여름 가족 고기 기도 영업 맛집 흑돼...
3,사계해변,산방산 사진 바람 해변 용머리해안 자연 인스타 식당 거리 시간 계획 송악산 디저트 ...
4,섭지코지,성산읍 성산일출봉 바람 유채꽃 거리 산책 식당 자연 음식 동쪽 사진 성리 드라마 겨...
...,...,...
95,종달리수국길,사진 종달리해안도로 성산일출봉 동쪽 산책 계획 바람 바다 기도 식당 여름 자연 거리...
96,녹산로유채꽃길,벚꽃 유채꽃길 축제 유채꽃 조랑말체험공원 노란 프라자 사진 거리 계획 바람 관광객 ...
97,스누피가든,정원 기념품 만화 캐릭터 찰리 산책 사진 거리 겨울 자연 계획 관광지 여름 체험 야...
98,제주센트럴파크,센트럴파크 한세상 거리 계획 인천 세대 인천광역시 산책 시장 사업 경쟁률 디저트 분...


In [108]:
loc_num=27
print(test_df['source'][loc_num])
test_df['keyword'][loc_num]

삼양해수욕장


'해수욕장 해변 모래 사진 디저트 바람 여름 거리 산책 바다 커피 삼양동 기도 영업 검은 시간 관광객 맛집 서핑 음식 삼양해수욕장 식당 가족 베이커리 일요일 계획 아메리카노 고기 가격 인기 자연 의자 건물 지도 풍경 창가 동영상 겨울 인테리어 냄새 케이크 관절염 검은색 버스 엘리베이터 아침 함덕해수욕장 메뉴판 매력 매주 구름 선택 강습 동쪽 치즈 영상 요리 수영 창문 흑돼지 신발 모래찜질 해수욕 신경통 나무 운전 먹기 음료 김치 동문시장 택시 경험 모래사장 좌석 고양이 소고기 입구 배달 시내 재미 사랑 취향 맛있어요 체험 언니 운동 김밥 후회 물놀이 철분 힐링 일몰 맨발 추억 낚시 놀이 식사 추워서 재료 맛있어'

In [109]:
import json

In [119]:
key_list_df =test_df.copy()

In [120]:
key_list_df['keyword'] = key_list_df['keyword'].apply(lambda x : x.split(' '))

In [121]:
key_list_df

,source,keyword
0,아르떼뮤지엄,"[미디어아트, 미디어, 전시관, 강릉시, 토끼, 경험, 입장권, 거울, 음악, 박람..."
1,산방산,"[유채꽃, 용머리해안, 바람, 사진, 거리, 식당, 용머리, 음식, 계획, 기도, ..."
2,함덕해수욕장,"[해수욕장, 식당, 바람, 산책, 해변, 음식, 거리, 사진, 여름, 가족, 고기,..."
3,사계해변,"[산방산, 사진, 바람, 해변, 용머리해안, 자연, 인스타, 식당, 거리, 시간, ..."
4,섭지코지,"[성산읍, 성산일출봉, 바람, 유채꽃, 거리, 산책, 식당, 자연, 음식, 동쪽, ..."
...,...,...
95,종달리수국길,"[사진, 종달리해안도로, 성산일출봉, 동쪽, 산책, 계획, 바람, 바다, 기도, 식..."
96,녹산로유채꽃길,"[벚꽃, 유채꽃길, 축제, 유채꽃, 조랑말체험공원, 노란, 프라자, 사진, 거리, ..."
97,스누피가든,"[정원, 기념품, 만화, 캐릭터, 찰리, 산책, 사진, 거리, 겨울, 자연, 계획,..."
98,제주센트럴파크,"[센트럴파크, 한세상, 거리, 계획, 인천, 세대, 인천광역시, 산책, 시장, 사업..."


In [115]:

with open('visitJeju.json', 'r',encoding='UTF-8') as f:

    json_data = json.load(f)

json_data 

[{'call': '(+82) 064-728-1527',
  'charge': '유료,일반 입장료 : 1.000원 / 장애인 무료 (선박료 별도)',
  'content': '우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. 완만한 경사와 옥토, 풍부한 어장, 우도팔경 등 천혜의 자연조건을 갖춘 관광지로써 한해 약 200만 명의 관광객이 찾는 제주의 대표적인 부속섬이다.?성산항과 종달항에서 우도가는 배를 탈 수 있는데 어디서 출발하든 15분 정도 소요된다. 섬의 길이는 3.8km, 둘레는 17km. 쉬지 않고 걸으면 3~4시간 걸리는 거리지만, 대부분의 관광객은 버스나 자전거, 미니 전기차를 타고 유명한 관광지 위주로 돌아본다.?검멀레해변이나 우도봉, 홍조단괴해변, 하고수동해변 등 유명한 관광지 1-2개를 둘러보고, 카페나 음식점에서 휴식을 즐겨도 대략 3-4시간 정도 소요된다. 여유있게 우도를 즐기고 싶다면 오전 아침배를 타고 들어가 오후 배를 타고 나와 하루종일 우도에 머물러 보는것도 좋다. 단, 기상에 따라 배 운항여부가 달라질수 있으니 우도 여행일정을 짜는데는 기상조건을 필히 확인해야한다.? 우도를 찾는 관광객은 홍조단괴해변, 우도봉, 검멀레 해변을 주로 찾는다. 홍조단괴해변은 산호해변으로도 불렸는데, 백사장을 이룬 하얀 알갱이가 산호가 아닌 홍조류가 딱딱하게 굳어 알갱이처럼 부서지면서 만들어진 것이 밝혀지면서 홍조단괴해변으로 부르며, 홍조류로 이뤄진 백사장은 세계에서 드물어 보호하고 있다.?너른 백사장과 아름다운 바다색으로 유명한 하고수동해수욕장도 있다. 경사가 완만한 천진동 코스와 경치가 멋진검멀레 해안코스가 있으며, 우도봉에 올라 우도의 전경을 바라볼 수도 있다. 자연 절경 이외에도 바다낚시, 자전거 하이킹, 잠수함과 유람선 등을 통해 여행의 재미를 더하고 있다.?※ 우도 외부차량(렌터카, 전세버스) 반입 제한 조치는 2022년 7월 31일까지로 연장되었다.(단, 1~3급 장애인과 만 65세 이상 노약자, 임산부, 

In [118]:
json_data[0]

{'call': '(+82) 064-728-1527',
 'charge': '유료,일반 입장료 : 1.000원 / 장애인 무료 (선박료 별도)',
 'content': '우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. 완만한 경사와 옥토, 풍부한 어장, 우도팔경 등 천혜의 자연조건을 갖춘 관광지로써 한해 약 200만 명의 관광객이 찾는 제주의 대표적인 부속섬이다.?성산항과 종달항에서 우도가는 배를 탈 수 있는데 어디서 출발하든 15분 정도 소요된다. 섬의 길이는 3.8km, 둘레는 17km. 쉬지 않고 걸으면 3~4시간 걸리는 거리지만, 대부분의 관광객은 버스나 자전거, 미니 전기차를 타고 유명한 관광지 위주로 돌아본다.?검멀레해변이나 우도봉, 홍조단괴해변, 하고수동해변 등 유명한 관광지 1-2개를 둘러보고, 카페나 음식점에서 휴식을 즐겨도 대략 3-4시간 정도 소요된다. 여유있게 우도를 즐기고 싶다면 오전 아침배를 타고 들어가 오후 배를 타고 나와 하루종일 우도에 머물러 보는것도 좋다. 단, 기상에 따라 배 운항여부가 달라질수 있으니 우도 여행일정을 짜는데는 기상조건을 필히 확인해야한다.? 우도를 찾는 관광객은 홍조단괴해변, 우도봉, 검멀레 해변을 주로 찾는다. 홍조단괴해변은 산호해변으로도 불렸는데, 백사장을 이룬 하얀 알갱이가 산호가 아닌 홍조류가 딱딱하게 굳어 알갱이처럼 부서지면서 만들어진 것이 밝혀지면서 홍조단괴해변으로 부르며, 홍조류로 이뤄진 백사장은 세계에서 드물어 보호하고 있다.?너른 백사장과 아름다운 바다색으로 유명한 하고수동해수욕장도 있다. 경사가 완만한 천진동 코스와 경치가 멋진검멀레 해안코스가 있으며, 우도봉에 올라 우도의 전경을 바라볼 수도 있다. 자연 절경 이외에도 바다낚시, 자전거 하이킹, 잠수함과 유람선 등을 통해 여행의 재미를 더하고 있다.?※ 우도 외부차량(렌터카, 전세버스) 반입 제한 조치는 2022년 7월 31일까지로 연장되었다.(단, 1~3급 장애인과 만 65세 이상 노약자, 임산부, 만 6

In [134]:
for dic in json_data:
    
    if dic['source'] in key_dict:
        dic['keyword'] = key_dict[dic['source']]
    
    else:
        dic['keyword'] = ""

In [141]:
json_data

{'call': '(+82) 064-728-3394',
 'charge': '',
 'content': "제주도의 동쪽에 위치하고 있는 마을인 '월정리'는 '달이 머문다'는 뜻의 이름을 가진 서정적인 풍경의 마을이다. 아름다운 에메랄드 빛 바다가 한 폭의 그림처럼 펼쳐져 있고, 그 위에는 밝은 달이 비친다. 풍경화처럼 아름다운 월정리 해변을 방문한 여행객들은 저마다 다양한 방법으로 해변의 경치를 만끽한다. 특히 수심이 얕은 편이기 때문에 아이를 동반한 가족들이 물놀이를 즐기기 좋다. ? 월정리의 아름다운 풍광이 여행객들 사이에서 점점 유명해질수록 월정리의 해변을 방문하는 사람들이 더욱 늘어났고, 그에 맞춰 다양한 식당과 카페, 숙박시설 등이 많이 들어서 있다. 몇몇 카페는 사람들이 바다를 보며 쉬어갈 수 있도록 의자를 두었는데, 이 의자에 앉아 찍은 사진이 유명해 지면서 하나의 포토스팟이 되기도 했다. 월정리엔 카메라를 들고 제주도 여행의 추억을 담는 사람들의 모습을 쉽게 볼 수 있다. ? 그 밖에 서핑, 스노클링, 카약 등 다양한 수상레포츠를 즐기며 보다 더 역동적으로 해변에서의 즐거움을 만끽하는 사람들도 있다. 월정리 해변은 일정한 높이의 파도가 지속적으로 들어오기 때문에 서핑을 하기에 좋아 서퍼들이 즐겨 찾는다. ?? 제주도 올레길 20코스 '김녕-하도 올레'와 ‘김녕-월정 지질트레일 코스’의 일부여서, ?뚜벅이 여행객들이 걷다가 바다의 아름다운 풍경과 시원한 바다 내음을 맡으며 쉴 수 있는 아름다운 해변이다.?",
 'convenience': '공용주차장,화장실,편의점,음료대,유도 및 안내시설',
 'detail_content': '',
 'difficult': '',
 'etc_property': '',
 'img': 'https://api.cdn.visitjeju.net/photomng/thumbnailpath/201804/30/d218b9b6-a3d2-4f64-8f93-fafcb4f9278b.jpg',
 'purpose': '',
 'purpose

In [142]:
with open('visit_result.json','w', encoding='utf-8') as f:
    json.dump(json_data, f, indent="\t", ensure_ascii=False)

In [145]:
key_list =key_list_df.keyword.sum()

In [154]:
print(len(key_list))
print(len(set(key_list)))

10000
1791


In [147]:
from collections import Counter

In [149]:
count = Counter(key_list)

len(count.most_common())

1791

In [152]:
key_data = [w for w,c in count.most_common()]

In [155]:
key_df = pd.DataFrame(data=key_data, columns=['keyword'])
key_df.to_csv('keyword.csv',encoding='UTF-8',index=False)